In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split 

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_confusion_matrix

In [32]:
diagnosis_admissions_table = pd.read_table('AdmissionsDiagnosesCorePopulatedTable_100_new.txt')
diagnosis_admissions_table['PatientID'] =  diagnosis_admissions_table['PatientID'] + '_' + diagnosis_admissions_table['AdmissionID'].astype(str)


In [33]:
diagnosis_admissions_table

,PatientID,AdmissionID,PrimaryDiagnosisCode,PrimaryDiagnosisDescription
0,80AC01B2-BD55-4BE0-A59A-4024104CF4E9_2,2,M01.X,Direct infection of joint in infectious and pa...
1,80AC01B2-BD55-4BE0-A59A-4024104CF4E9_3,3,D65,Disseminated intravascular coagulation [defibr...
2,80AC01B2-BD55-4BE0-A59A-4024104CF4E9_4,4,C92.1,"Chronic myeloid leukemia, BCR/ABL-positive"
3,80AC01B2-BD55-4BE0-A59A-4024104CF4E9_5,5,M05.51,Rheumatoid polyneuropathy with rheumatoid arth...
4,6A57AC0C-57F3-4C19-98A1-51135EFBC4FF_1,1,C91.00,Acute lymphoblastic leukemia not having achiev...
...,...,...,...,...
367,2A8772FE-61DB-483E-B6BF-6C0A74BA9C2A_3,3,H15,Disorders of sclera
368,C65A4ADE-112E-49E4-B72A-0DED22C242ED_1,1,M05.27,Rheumatoid vasculitis with rheumatoid arthriti...
369,C65A4ADE-112E-49E4-B72A-0DED22C242ED_2,2,N16,Renal tubulo-interstitial disorders in disease...
370,C65A4ADE-112E-49E4-B72A-0DED22C242ED_3,3,D35.2,Benign neoplasm of pituitary gland


In [34]:
labs_table = pd.read_table('LabsCorePopulatedTable_100_new.txt')
labs_table['PatientID'] =  labs_table['PatientID'] + '_' + labs_table['AdmissionID'].astype(str)


In [35]:
labs_table

,PatientID,AdmissionID,LabName,LabValue,LabUnits,LabDateTime
0,1A8791E3-A61C-455A-8DEE-763EB90C9B2C_1,1,URINALYSIS: RED BLOOD CELLS,1.8,rbc/hpf,1992-07-01 01:36:17.910
1,1A8791E3-A61C-455A-8DEE-763EB90C9B2C_1,1,METABOLIC: GLUCOSE,103.3,mg/dL,1992-06-30 09:35:52.383
2,1A8791E3-A61C-455A-8DEE-763EB90C9B2C_1,1,CBC: MCH,35.8,pg,1992-06-30 03:50:11.777
3,1A8791E3-A61C-455A-8DEE-763EB90C9B2C_1,1,METABOLIC: CALCIUM,8.9,mg/dL,1992-06-30 12:09:46.107
4,1A8791E3-A61C-455A-8DEE-763EB90C9B2C_1,1,CBC: RED BLOOD CELL COUNT,4.8,m/cumm,1992-07-01 01:31:08.677
...,...,...,...,...,...,...
111478,F0B53A2C-98CA-415D-B928-E3FD0E52B22A_6,6,CBC: HEMOGLOBIN,13.6,gm/dl,2012-04-21 03:23:50.883
111479,F0B53A2C-98CA-415D-B928-E3FD0E52B22A_6,6,CBC: MCH,26.2,pg,2012-04-21 03:18:44.127
111480,F0B53A2C-98CA-415D-B928-E3FD0E52B22A_6,6,METABOLIC: CHLORIDE,104.4,mmol/L,2012-04-21 02:58:57.840
111481,F0B53A2C-98CA-415D-B928-E3FD0E52B22A_6,6,METABOLIC: GLUCOSE,63.3,mg/dL,2012-04-21 02:50:04.503


In [36]:

diagnosis_admissions_table.sort_values('PatientID', inplace=True)
total_diagnoses_specific = diagnosis_admissions_table['PrimaryDiagnosisDescription'] # for with details (ex: Protozoal diseases complicating pregnancy, first trimester)


In [37]:
total_diagnoses_specific

180    Diseases of the skin and subcutaneous tissue c...
181           Type 2 diabetes mellitus with hypoglycemia
182    Atherosclerosis of bypass graft of coronary ar...
183    Pre-existing diabetes mellitus, type 2, in pre...
184    Sedative, hypnotic or anxiolytic abuse with se...
                             ...                        
227                                 Hepatorenal syndrome
228                   Malignant neoplasm of pineal gland
125                                    Vascular dementia
126                      Hairy cell leukemia, in relapse
127    Malignant neoplasm of overlapping sites of rec...
Name: PrimaryDiagnosisDescription, Length: 372, dtype: object

In [38]:
grouped_labs = labs_table.groupby(['PatientID', 'LabName']).aggregate(np.mean)
og_labs = grouped_labs.reset_index()

In [39]:
og_labs

,PatientID,LabName,AdmissionID,LabValue
0,016A590E-D093-4667-A5DA-D68EA6987D93_1,CBC: ABSOLUTE LYMPHOCYTES,1,21.050000
1,016A590E-D093-4667-A5DA-D68EA6987D93_1,CBC: ABSOLUTE NEUTROPHILS,1,69.300000
2,016A590E-D093-4667-A5DA-D68EA6987D93_1,CBC: BASOPHILS,1,0.125000
3,016A590E-D093-4667-A5DA-D68EA6987D93_1,CBC: EOSINOPHILS,1,0.275000
4,016A590E-D093-4667-A5DA-D68EA6987D93_1,CBC: HEMATOCRIT,1,43.500000
...,...,...,...,...
12998,FFCDECD6-4048-4DCB-B910-1218160005B3_3,METABOLIC: TOTAL PROTEIN,3,7.080000
12999,FFCDECD6-4048-4DCB-B910-1218160005B3_3,URINALYSIS: PH,3,5.857143
13000,FFCDECD6-4048-4DCB-B910-1218160005B3_3,URINALYSIS: RED BLOOD CELLS,3,2.137500
13001,FFCDECD6-4048-4DCB-B910-1218160005B3_3,URINALYSIS: SPECIFIC GRAVITY,3,1.000000


In [40]:
x = og_labs.pivot(index='PatientID', columns='LabName', values='LabValue').sort_index()

In [41]:
x_dub = x

In [42]:
x

LabName,CBC: ABSOLUTE LYMPHOCYTES,CBC: ABSOLUTE NEUTROPHILS,CBC: BASOPHILS,CBC: EOSINOPHILS,CBC: HEMATOCRIT,CBC: HEMOGLOBIN,CBC: LYMPHOCYTES,CBC: MCH,CBC: MCHC,CBC: MEAN CORPUSCULAR VOLUME,...,METABOLIC: CHLORIDE,METABOLIC: CREATININE,METABOLIC: GLUCOSE,METABOLIC: POTASSIUM,METABOLIC: SODIUM,METABOLIC: TOTAL PROTEIN,URINALYSIS: PH,URINALYSIS: RED BLOOD CELLS,URINALYSIS: SPECIFIC GRAVITY,URINALYSIS: WHITE BLOOD CELLS
PatientID,,,,,,,,,,,,,,,,,,,,,
016A590E-D093-4667-A5DA-D68EA6987D93_1,21.050000,69.300000,0.125000,0.275000,43.500000,13.060000,3.650000,29.283333,32.280000,83.000000,...,108.400000,0.820000,98.540000,4.525000,144.520000,6.666667,6.250000,1.575000,1.0,3.500000
016A590E-D093-4667-A5DA-D68EA6987D93_2,29.850000,72.360000,0.080000,0.400000,39.380000,14.875000,3.500000,34.533333,33.600000,85.475000,...,103.720000,0.814286,102.580000,4.920000,137.380000,6.750000,6.142857,2.100000,1.0,2.400000
016A590E-D093-4667-A5DA-D68EA6987D93_3,24.240000,67.566667,0.077778,0.322222,35.712500,14.350000,2.850000,29.228571,32.211111,85.460000,...,97.388889,0.877778,97.012500,4.012500,139.340000,7.525000,5.750000,0.816667,1.0,2.625000
016A590E-D093-4667-A5DA-D68EA6987D93_4,22.571429,70.925000,0.083333,0.233333,41.200000,15.087500,3.733333,32.255556,33.257143,84.820000,...,100.657143,0.900000,106.550000,4.337500,137.583333,7.900000,6.287500,1.133333,1.0,3.911111
016A590E-D093-4667-A5DA-D68EA6987D93_5,28.383333,66.588889,0.166667,0.357143,44.250000,13.155556,3.280000,30.528571,32.944444,80.066667,...,100.775000,0.912500,108.870000,4.580000,139.316667,7.480000,6.000000,1.888889,1.0,3.225000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FE0B9B59-1927-45B7-8556-E079DC1DE30A_2,25.566667,67.816667,0.100000,0.400000,42.442857,13.280000,3.022222,29.150000,33.400000,80.840000,...,103.800000,0.844444,106.133333,4.525000,137.371429,8.122222,5.975000,1.188889,1.0,2.171429
FE0B9B59-1927-45B7-8556-E079DC1DE30A_3,25.383333,69.800000,0.122222,0.300000,47.200000,14.800000,2.528571,27.511111,32.366667,84.875000,...,100.375000,0.922222,103.200000,3.900000,142.312500,7.111111,5.585714,1.757143,1.0,3.750000
FFCDECD6-4048-4DCB-B910-1218160005B3_1,25.886667,68.686667,0.109091,0.357143,42.676923,14.309091,2.587500,29.636364,33.557143,87.193333,...,100.208333,0.900000,103.516667,4.216667,137.850000,7.293750,5.885714,2.000000,1.0,2.807692


In [43]:
disease = input() # ex: neoplasm

neoplasm


In [44]:
y = []

for diagnosis in total_diagnoses_specific:
    if disease in diagnosis:
        y.append(True)
    else: 
        y.append(False)

In [45]:
y

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 

In [46]:
x.fillna(-100, inplace=True)


In [47]:
x_train_lr, x_test_lr, y_train_lr, y_test_lr = train_test_split(x, y, test_size=.2)

model_lr = LogisticRegression()
model_lr.fit(x_train_lr, y_train_lr)
model_lr.score(x_test_lr, y_test_lr)

/Users/ishanidas/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8266666666666667

In [48]:
x_test_lr

LabName,CBC: ABSOLUTE LYMPHOCYTES,CBC: ABSOLUTE NEUTROPHILS,CBC: BASOPHILS,CBC: EOSINOPHILS,CBC: HEMATOCRIT,CBC: HEMOGLOBIN,CBC: LYMPHOCYTES,CBC: MCH,CBC: MCHC,CBC: MEAN CORPUSCULAR VOLUME,...,METABOLIC: CHLORIDE,METABOLIC: CREATININE,METABOLIC: GLUCOSE,METABOLIC: POTASSIUM,METABOLIC: SODIUM,METABOLIC: TOTAL PROTEIN,URINALYSIS: PH,URINALYSIS: RED BLOOD CELLS,URINALYSIS: SPECIFIC GRAVITY,URINALYSIS: WHITE BLOOD CELLS
PatientID,,,,,,,,,,,,,,,,,,,,,
F0B53A2C-98CA-415D-B928-E3FD0E52B22A_2,28.550000,73.400000,0.066667,0.175000,36.866667,15.100000,2.050000,31.400000,34.750000,87.875000,...,95.466667,0.975000,84.350000,4.333333,130.500000,8.366667,5.650000,2.400000,1.0,1.925000
1A40AF35-C6D4-4D46-B475-A15D84E8A9D5_3,24.480000,71.081818,0.110000,0.433333,38.033333,13.837500,2.311111,31.670000,34.357143,82.340000,...,99.320000,0.728571,104.350000,4.462500,138.811111,7.775000,6.370000,1.614286,1.0,2.855556
C242E3A4-E785-4DF1-A0E4-3B568DC88F2E_1,27.437500,66.954545,0.111111,0.325000,47.133333,14.777778,3.330000,30.627273,32.150000,84.653846,...,105.280000,0.960000,84.650000,4.322222,140.330000,7.911111,6.316667,1.571429,1.0,2.944444
9C75DF1F-9DA6-4C98-8F5B-E10BDC805ED0_1,20.933333,71.800000,0.120000,0.275000,37.475000,13.350000,1.740000,33.616667,34.800000,86.660000,...,100.583333,0.840000,106.383333,4.900000,143.014286,7.166667,6.080000,1.600000,1.0,2.225000
79A7BA2A-D35A-4CB8-A835-6BAA13B0058C_4,25.928571,70.633333,0.144444,0.322222,42.300000,14.040000,2.272727,29.850000,33.766667,85.691667,...,99.150000,0.846667,98.838462,4.792308,135.036364,7.316667,6.408333,1.871429,1.0,2.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36775002-9EC3-4889-AD4F-80DC6855C8D8_6,26.441667,71.460000,0.150000,0.372727,47.550000,13.633333,2.227273,29.641667,34.523077,85.000000,...,104.907692,0.790909,99.577778,4.077778,140.710000,7.336364,6.120000,2.343750,1.0,2.821429
25B786AF-0F99-478C-9CFA-0EA607E45834_7,24.571429,70.617647,0.130000,0.320000,43.753846,14.361538,2.305556,28.500000,35.546154,81.286667,...,102.836364,0.792857,94.107143,4.581250,138.546667,7.084615,5.880000,1.830769,1.0,2.723077
69CC25ED-A54A-4BAF-97E3-774BB3C9DED1_2,27.375000,71.050000,0.200000,0.233333,47.433333,13.740000,2.033333,29.766667,36.100000,85.000000,...,95.366667,0.950000,105.200000,4.350000,138.166667,8.566667,6.900000,2.433333,1.0,4.940000


In [49]:
y_test_lr

[False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True]

In [50]:
x_dub.drop(x_dub.index, inplace=True) # clearing df

In [51]:
x_dub

LabName,CBC: ABSOLUTE LYMPHOCYTES,CBC: ABSOLUTE NEUTROPHILS,CBC: BASOPHILS,CBC: EOSINOPHILS,CBC: HEMATOCRIT,CBC: HEMOGLOBIN,CBC: LYMPHOCYTES,CBC: MCH,CBC: MCHC,CBC: MEAN CORPUSCULAR VOLUME,...,METABOLIC: CHLORIDE,METABOLIC: CREATININE,METABOLIC: GLUCOSE,METABOLIC: POTASSIUM,METABOLIC: SODIUM,METABOLIC: TOTAL PROTEIN,URINALYSIS: PH,URINALYSIS: RED BLOOD CELLS,URINALYSIS: SPECIFIC GRAVITY,URINALYSIS: WHITE BLOOD CELLS
PatientID,,,,,,,,,,,,,,,,,,,,,


In [52]:
labs_names = list(x_dub)

In [53]:
labs_names

['CBC: ABSOLUTE LYMPHOCYTES',
 'CBC: ABSOLUTE NEUTROPHILS',
 'CBC: BASOPHILS',
 'CBC: EOSINOPHILS',
 'CBC: HEMATOCRIT',
 'CBC: HEMOGLOBIN',
 'CBC: LYMPHOCYTES',
 'CBC: MCH',
 'CBC: MCHC',
 'CBC: MEAN CORPUSCULAR VOLUME',
 'CBC: MONOCYTES',
 'CBC: NEUTROPHILS',
 'CBC: PLATELET COUNT',
 'CBC: RDW',
 'CBC: RED BLOOD CELL COUNT',
 'CBC: WHITE BLOOD CELL COUNT',
 'METABOLIC: ALBUMIN',
 'METABOLIC: ALK PHOS',
 'METABOLIC: ALT/SGPT',
 'METABOLIC: ANION GAP',
 'METABOLIC: AST/SGOT',
 'METABOLIC: BILI TOTAL',
 'METABOLIC: BUN',
 'METABOLIC: CALCIUM',
 'METABOLIC: CARBON DIOXIDE',
 'METABOLIC: CHLORIDE',
 'METABOLIC: CREATININE',
 'METABOLIC: GLUCOSE',
 'METABOLIC: POTASSIUM',
 'METABOLIC: SODIUM',
 'METABOLIC: TOTAL PROTEIN',
 'URINALYSIS: PH',
 'URINALYSIS: RED BLOOD CELLS',
 'URINALYSIS: SPECIFIC GRAVITY',
 'URINALYSIS: WHITE BLOOD CELLS']

In [54]:
new_pat_labs = {}
counter = 0
all_labs = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

for lab_name in labs_names:
    new_pat_labs[lab_name] = all_labs[counter]
    counter+=1

In [55]:
new_pat_labs

{'CBC: ABSOLUTE LYMPHOCYTES': 1,
 'CBC: ABSOLUTE NEUTROPHILS': 1,
 'CBC: BASOPHILS': 1,
 'CBC: EOSINOPHILS': 1,
 'CBC: HEMATOCRIT': 1,
 'CBC: HEMOGLOBIN': 1,
 'CBC: LYMPHOCYTES': 1,
 'CBC: MCH': 1,
 'CBC: MCHC': 1,
 'CBC: MEAN CORPUSCULAR VOLUME': 1,
 'CBC: MONOCYTES': 1,
 'CBC: NEUTROPHILS': 1,
 'CBC: PLATELET COUNT': 1,
 'CBC: RDW': 1,
 'CBC: RED BLOOD CELL COUNT': 1,
 'CBC: WHITE BLOOD CELL COUNT': 1,
 'METABOLIC: ALBUMIN': 1,
 'METABOLIC: ALK PHOS': 1,
 'METABOLIC: ALT/SGPT': 1,
 'METABOLIC: ANION GAP': 1,
 'METABOLIC: AST/SGOT': 1,
 'METABOLIC: BILI TOTAL': 1,
 'METABOLIC: BUN': 1,
 'METABOLIC: CALCIUM': 1,
 'METABOLIC: CARBON DIOXIDE': 1,
 'METABOLIC: CHLORIDE': 1,
 'METABOLIC: CREATININE': 1,
 'METABOLIC: GLUCOSE': 1,
 'METABOLIC: POTASSIUM': 1,
 'METABOLIC: SODIUM': 1,
 'METABOLIC: TOTAL PROTEIN': 1,
 'URINALYSIS: PH': 1,
 'URINALYSIS: RED BLOOD CELLS': 1,
 'URINALYSIS: SPECIFIC GRAVITY': 1,
 'URINALYSIS: WHITE BLOOD CELLS': 1}

In [56]:
x_dub = x_dub.append(new_pat_labs, ignore_index=True)

In [57]:
x_dub

LabName,CBC: ABSOLUTE LYMPHOCYTES,CBC: ABSOLUTE NEUTROPHILS,CBC: BASOPHILS,CBC: EOSINOPHILS,CBC: HEMATOCRIT,CBC: HEMOGLOBIN,CBC: LYMPHOCYTES,CBC: MCH,CBC: MCHC,CBC: MEAN CORPUSCULAR VOLUME,...,METABOLIC: CHLORIDE,METABOLIC: CREATININE,METABOLIC: GLUCOSE,METABOLIC: POTASSIUM,METABOLIC: SODIUM,METABOLIC: TOTAL PROTEIN,URINALYSIS: PH,URINALYSIS: RED BLOOD CELLS,URINALYSIS: SPECIFIC GRAVITY,URINALYSIS: WHITE BLOOD CELLS
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [58]:
model_lr.predict(x_dub)[0]

True